In [1]:
import math
from math import pi as PI
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from matplotlib import animation, pyplot as plt
from helper import d_beta, d_beta_numeric, beta, simple_gaussian, broken_sigmoid, minimum_separation,\
    collision_trajectory, v2sp, sp2v, sp2a, rotate, theta, d_theta, d_theta_numeric, beta, d_beta, d_beta_numeric,\
    psi, d_psi, d_psi_numeric, phi
from simulator import Simulation, Agent
from models import Model
import pandas as pd

In [3]:
'''Simulator debug'''
%matplotlib qt
p0 = np.array(sim.p[0])
p1 = np.array(sim.p[1])
# plt.scatter(p0[:,0], p0[:,1], s=1)
# plt.scatter(p1[:,0], p1[:,1], s=1)
# len(sim.t)

In [3]:
'''Simulation'''
# (id, goal_id, w, p0, v0, a0, s, phi, d_phi, dd_phi, d_s, p_spd, model_names, model_args, constant, ref=[0,1])
%matplotlib qt
agents = []
models = ['approach_stationary_goal']
args = [{'p_spd': 1.3, 't_relax': 3, 'b': 3.25, 'k_g': 7.5, 'c_1': 0.4, 'c_2': 0.4}]
#  'optical_ratio_model'   {'k_s': 0.1, 'k_h': 0.05, 'c': 1}
# Goal
agents.append(Agent(0, None, None, [5, 5], [0, 0], [0, 0], None, None, 0, 0, 0, 1.3, ['null'], [{}], True))
# Agent 1
agents.append(Agent(1, 0, 0.5, [0, -5], [0, 1], [0, 0], None, None, 0, 0, 0, 1.3, models, args, False))
# Agent 2
# agents.append(Agent(2, None, 0.5, [0, 10], [1, -1], [0, 0], None, None, 0, 0, 0, 1.3, ['null'], [{}], True))
sim = Simulation(agents, 100)
sim.simulate(10)
sim.play(interval=None, save=True)

In [4]:
# Debug helper functions
%matplotlib qt

Hz = 100
w = 2
ref = [1, 0]
p0, p1, v0, v1 = collision_trajectory(45, 'b', spd1=2, w=w, r_min=w, Hz=Hz, animate=False)

# d_psi1 = d_psi_numeric(p0, p1, Hz, ref=ref)
# d_psi2 = d_psi(p0, p1, v0, v1, ref=ref)
# plt.plot(d_psi1)
# plt.plot(d_psi2)

a0 = np.zeros_like(p0)
d_betas1 = d_beta_numeric(p0, p1, v0, Hz)
d_betas2 = d_beta(p0, p1, v0, v1, a0)

plt.plot(d_betas1)
plt.plot(d_betas2)


# data = np.concatenate((p0, p1, v0, v1, a0, np.expand_dims(d_betas1, axis=1), np.expand_dims(d_betas2, axis=1)), axis=1)

# df = pd.DataFrame(data=data, columns=['p0x', 'p0y', 'p1x', 'p1y', 'v0x', 'v0y', 'v1x', 'v1y', 'a0x', 'a0y', 'd_beta_numeric', 'd_beta_analytic'])
# df.to_csv("d_beta.csv", index=False)


D:\OneDrive\Dissertation\simulations\helper.py:264: RuntimeWarning: divide by zero encountered in double_scalars
  return - numerator / denominator


In [5]:
# Run the simulation
sim.p[0]





(array([1.41421356]), array([0.78539816]))

In [23]:
# Plot
%matplotlib qt
Hz = 100
w = 0.5
traj0, traj1, v0, v1 = collision_trajectory(20, 'f', spd1=1.3, w=w, r_min=w, Hz=Hz, animate=True)
a0 = np.zeros_like(traj0)
thetas = theta(traj0, traj1, w)
d_betas = np.absolute(d_beta_numeric(traj0, traj1, v0, Hz=Hz))
d_thetas = d_theta(traj0, traj1, v0, v1, 2)
RRE = d_thetas / thetas
end = np.argmax(np.absolute(d_betas))

plt.figure()
# Rate of bearing change and rate of expansion
plt.plot(d_betas, label='d_betas numerical')
plt.plot(RRE, label='RRE')
plt.legend()

plt.figure()
# Critical rate of bearing change as a function of (relative) rate of expansion
c = 10
ratio = (RRE[:end] + c) / (d_betas[:end] + c) - 1
plt.plot(ratio)



In [17]:
# Plot rate of bearing change as a function of (relative) rate of expansion at different angle of interception
%matplotlib qt
Hz = 100
for i in range(10, 80, 1):
    traj0, traj1, v0, v1 = collision_trajectory(i, 'f', Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
#     plt.scatter(np.absolute(d_beta[: end]), d_theta[: end], marker='.', s=1)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)
    

In [15]:
# Plot rate of bearing change as a function of (relative) rate of expansion at different speed
%matplotlib qt
Hz = 100
for i in range(8, 16, 1):
    i = i / 10.0
    traj0, traj1, v0, v1 = collision_trajectory(45, 'f', spd1=i, Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)

In [18]:
# Plot rate of bearing change as a function of (relative) rate of expansion with different width
%matplotlib qt
Hz = 100
for i in range(2, 20, 1):
    i = i / 10.0
    traj0, traj1, v0, v1 = collision_trajectory(45, 'f', w=i, Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)